In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

In [8]:
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [9]:
eko_dataframe = pd.read_csv("./mfcc_csv/eko.csv")
mic_dataframe = pd.read_csv("./mfcc_csv/mic.csv")

In [89]:
def fold(sound_dataframe, sets_loc):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 
        try:
            set_dataframe = pd.DataFrame()
            for i, line in enumerate(Lines):
                A = sound_dataframe[(sound_dataframe['73'] == (Lines[i].strip()+".json"))]
                set_dataframe = pd.DataFrame.append(set_dataframe,A)
        except:
            set_dataframe = pd.DataFrame()
            for i, line in enumerate(Lines):
                A = sound_dataframe[(sound_dataframe[73] == (Lines[i].strip()+".json"))]
                set_dataframe = pd.DataFrame.append(set_dataframe,A)
            
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:6]))
    test_1 = shuffle(big[6])
    fold_2 = shuffle(pd.concat(big[1:7]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4], big[5], big[6]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4], big[5], big[6]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[5], big[6]]))
    test_5 = shuffle(big[3])
    fold_6 = shuffle(pd.concat([big[0],big[2],big[1],big[3], big[5], big[6]]))
    test_6 = shuffle(big[4])
    fold_7 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[3], big[6]]))
    test_7 = shuffle(big[5])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5, fold_6, test_6, fold_7,test_7

In [90]:
def SVM(fold_list):
    list_train = []
    list_test = []
    for i in range(0,len(fold_list),2):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:72]
        y_train = fold.iloc[:,72]
        X_test = test.iloc[:,:72]
        y_test = test.iloc[:,72]


        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)



        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)
        


        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train

In [ ]:
def hyperparameter_tuning(fold,test):
    X_train = fold.iloc[:,:72]
    y_train = fold.iloc[:,72]    
    C_range = np.logspace(-2, 3, 10)
    gamma_range = np.logspace(-10, 3, 20, base=2)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(n_splits=5)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv, verbose=3)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    X_test = test.iloc[:,:72]
    X_test = scaler.transform(X_test)
    y_pred = grid.predict(X_test)
    a = accuracy_score(test.iloc[:,72],y_pred)
    return a

In [ ]:
tune_acc_eko = []
for i in range(0,len(fold_list),2):
#     print(i)
    tune_acc_eko.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

In [ ]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_dataframe,sets_loc)

In [ ]:
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7]

In [ ]:
mic_test, mic_train = SVM(fold_list)

In [ ]:
mic_test

In [ ]:
tune_acc_mic = []
for i in range(0,len(fold_list),2):
#     print(i)
    tune_acc_mic.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

### Wheeze test

In [ ]:


import pickle

In [ ]:
loaded_model = pickle.load(open('./Wheeze.model', 'rb'))

In [ ]:
f1 = pd.read_csv('./f1.csv')

In [ ]:
list_test = []
def test_wheeze(test_set):
    X_test = test_set.iloc[:,:72]
    y_test = test_set.iloc[:,72]
    scaler = StandardScaler()
    scaler.fit(f1.iloc[:,:72])
    X_test = scaler.transform(X_test)
    X_test = pd.DataFrame(X_test)
    return accuracy_score(y_test,loaded_model.predict(X_test))

In [ ]:
acc_eko_wheeze = []
for i in range(0,len(fold_list),2):
#     print(i)
    acc_eko_wheeze.append(test_wheeze(fold_list[i+1]))

In [ ]:
acc_eko_wheeze

In [ ]:
acc_eko_wheeze.append(np.mean(acc_eko_wheeze))

acc_eko_wheeze.append(np.std(acc_eko_wheeze))

In [ ]:
pd.DataFrame(acc_eko_wheeze)

In [ ]:
acc_mic_wheeze = []
for i in range(0,len(fold_list),2):
#     print(i)
    acc_mic_wheeze.append(test_wheeze(fold_list[i+1]))

In [ ]:
acc_mic_wheeze.append(np.mean(acc_mic_wheeze))

acc_mic_wheeze.append(np.std(acc_mic_wheeze))

In [ ]:
pd.DataFrame(acc_mic_wheeze)

### 144 SVM


In [14]:
eko_144_dataframe = pd.read_csv("./144_mfcc_csv/eko.csv")

In [15]:
mic_144_dataframe = pd.read_csv("./144_mfcc_csv/mic.csv")

In [41]:
(eko_144_dataframe['72'] == mic_144_dataframe['72']).value_counts()

True    4843
Name: 72, dtype: int64

In [36]:
mic_144_dataframe

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-3.432589,-9.773512,-6.549938,0.678330,-5.894518,0.697558,13.498579,-3.030802,-2.117977,-9.468413,...,8.061950,5.406046,14.957045,7.568259,8.809546,12.028821,9.837308,6.569900,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
1,-3.478034,-11.049839,-3.842209,1.642960,-4.947368,1.345134,14.291431,-1.719351,-3.237112,-7.766280,...,7.395209,5.305053,15.857155,6.044535,6.901877,10.599804,8.341879,6.842846,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
2,-3.305058,-11.059169,-7.866313,1.751684,-4.892396,1.403097,13.946215,-3.745874,-2.762307,-8.164956,...,7.621772,5.613023,15.165219,7.114400,6.636801,10.990638,9.846593,6.195352,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
3,-3.624479,-11.491781,-5.848464,0.445673,-5.009919,0.664603,9.352459,-2.461240,1.979536,-7.535662,...,9.791298,5.556102,12.211047,6.690642,7.140235,11.325418,11.565703,6.758375,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
4,-3.527560,-11.842173,-6.543797,1.719556,-3.507234,0.452993,10.877880,-1.091079,1.583291,-6.830845,...,8.358533,5.584833,12.688793,6.325704,9.693023,11.022108,11.129185,7.762274,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,-1.707336,-12.343208,6.110630,0.752774,14.427552,-10.183993,16.413902,-4.055749,15.492588,-7.702860,...,16.330696,13.549398,17.507429,6.894721,17.095067,10.810694,12.708401,7.509805,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json
4839,-1.528619,-13.455676,5.108312,5.486788,15.292221,-9.100983,18.525646,-1.718810,17.352399,-6.813103,...,16.004566,10.694211,19.230394,9.262316,18.670967,10.249268,9.755718,8.340064,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json
4840,-3.310805,-25.219895,1.888971,-0.614274,10.134691,-5.223078,10.222503,-5.202263,14.450614,-8.457116,...,12.689370,9.255334,12.950238,8.688386,17.485942,10.982377,11.644869,9.945285,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json
4841,-2.782620,-18.485031,3.324376,0.556895,10.864181,-6.011137,14.170535,-6.002723,15.268375,-2.514071,...,13.095762,9.086908,15.380179,8.827261,17.518220,7.774121,10.256598,9.209482,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json


In [51]:
Data_144 = pd.concat([eko_144_dataframe.iloc[:,:72],mic_144_dataframe.iloc[:,:72],mic_144_dataframe['72'],mic_144_dataframe['73'] ],axis = 1,ignore_index=True )

In [62]:
Data_144[144]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
4838    0.0
4839    0.0
4840    0.0
4841    0.0
4842    0.0
Name: 144, Length: 4843, dtype: float64

In [50]:
Data_144[144].value_counts()

0.0    2574
1.0    2269
Name: 144, dtype: int64

In [78]:
def fold144(sound_dataframe, sets_loc):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 

        set_dataframe = pd.DataFrame()
        for i, line in enumerate(Lines):
            A = sound_dataframe[(sound_dataframe[145] == (Lines[i].strip()+".json"))]
            set_dataframe = pd.DataFrame.append(set_dataframe,A)
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:6]))
    test_1 = shuffle(big[6])
    fold_2 = shuffle(pd.concat(big[1:7]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4], big[5], big[6]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4], big[5], big[6]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[5], big[6]]))
    test_5 = shuffle(big[3])
    fold_6 = shuffle(pd.concat([big[0],big[2],big[1],big[3], big[5], big[6]]))
    test_6 = shuffle(big[4])
    fold_7 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[3], big[6]]))
    test_7 = shuffle(big[5])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5, fold_6, test_6, fold_7,test_7

In [59]:
sets_loc = './Sets/'
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(Data_144,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7]

In [68]:
def SVM144(fold_list):
    list_train = []
    list_test = []
    for i in range(0,len(fold_list),2):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:144]
        y_train = fold.iloc[:,144]
        X_test = test.iloc[:,:144]
        y_test = test.iloc[:,144]


        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)



        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)
        


        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train

In [69]:
Data_144_test, Data_144_train = SVM144(fold_list)

In [70]:

Data_144_test

[0.5308056872037915,
 0.6623748211731044,
 0.5902864259028643,
 0.6185567010309279,
 0.6335149863760218,
 0.45703125,
 0.6158445440956651]

In [71]:
np.mean(Data_144_test)

0.5869163451117679

### Mean of eko and Mic

In [76]:
mean_Dataframe = pd.concat([eko_144_dataframe.iloc[:,:72]+mic_144_dataframe.iloc[:,:72] ,mic_144_dataframe['72'],mic_144_dataframe['73']],axis = 1,ignore_index=True)

In [88]:
mean_Dataframe

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-11.928713,16.048576,7.211864,-3.994284,-15.794262,-5.747138,10.768925,-2.382203,0.705776,-7.327418,...,21.072345,16.305670,24.475871,16.552353,17.299448,20.715844,18.062649,14.131335,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
1,-12.311058,12.768932,9.794439,-2.055621,-15.077798,-6.175074,11.073408,-0.765216,0.350876,-5.068849,...,20.517032,16.332159,24.467457,14.281034,16.263853,19.750515,16.682692,14.546865,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
2,-11.910596,13.398791,5.937259,-2.047984,-15.008789,-5.622323,12.034330,-1.721921,0.805769,-5.899443,...,20.443425,16.442701,23.914209,15.958499,15.953248,19.405361,18.129495,13.686211,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
3,-12.245382,13.435432,7.660490,-3.981835,-15.052877,-6.484086,5.945840,-1.843741,5.219039,-5.393585,...,23.295584,16.755704,21.209514,15.508630,16.258860,19.657638,19.436407,13.997897,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
4,-12.425746,11.979840,6.537419,-2.680382,-13.797291,-7.081280,7.593761,0.179397,5.405407,-4.742015,...,21.329497,16.490133,21.255579,14.250486,19.182954,19.592779,18.822347,14.917507,0.0,2019_11_06_Sabheela_F_46_154_82_N_HC_NA.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,-9.900400,10.939282,11.400618,2.923752,25.158692,-11.152744,6.369979,-4.578481,17.996667,-8.358923,...,29.440395,22.536971,31.286148,15.979962,27.210676,20.910081,21.575482,15.236486,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json
4839,-10.269867,6.598326,8.829967,7.332890,25.904973,-9.700215,8.030837,-4.114524,18.731161,-7.189370,...,29.324272,20.005829,32.975626,19.019947,27.893910,19.082834,19.581206,17.497466,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json
4840,-11.991669,-4.164566,7.658187,1.778502,19.288662,-6.367058,0.740590,-6.783289,15.608331,-9.964872,...,24.604034,16.981701,25.662829,17.994353,26.663763,20.264592,20.814344,18.492114,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json
4841,-11.381171,2.357272,8.821226,3.751851,21.645239,-5.483348,6.712880,-6.711098,15.136033,-5.384721,...,26.106785,18.092530,27.117588,17.741023,27.298868,17.775095,19.234526,17.707416,0.0,2019_11_22_PavaniKumari_F_22_164_59_N_HC_NA.json


In [91]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mean_Dataframe, sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7]

In [93]:
mean_eko_mic_test, mean_eko_mic_train = SVM(fold_list)

In [94]:
mean_eko_mic_test

[0.5,
 0.597997138769671,
 0.5765877957658779,
 0.6099656357388317,
 0.5735694822888283,
 0.35546875,
 0.5754857997010463]

In [95]:
np.mean(mean_eko_mic_test)

0.5412963717520365

In [96]:
np.std(mean_eko_mic_test)

0.0824826348641266